# SQL Basics: Join Types and Set Operations

## Join Types

### INNER JOIN
Returns only the matching rows between two tables based on a common key.

```sql
SELECT A.*, B.*
FROM TableA A
INNER JOIN TableB B
ON A.id = B.id;
```


# LEFT JOIN (or LEFT OUTER JOIN)
Returns all rows from the left table and matching rows from the right table. If no match, NULL values are returned for the right table.

```sql
SELECT A.*, B.*
FROM TableA A
LEFT JOIN TableB B
ON A.id = B.id;
```

# RIGHT JOIN (or RIGHT OUTER JOIN)
Returns all rows from the right table and matching rows from the left table. If no match, NULL values are returned for the left table.

```sql
SELECT A.*, B.*
FROM TableA A
RIGHT JOIN TableB B
ON A.id = B.id;

```

# FULL JOIN (or FULL OUTER JOIN)
Returns all rows from both tables, with NULLs for missing matches in either table.

```sql
SELECT A.*, B.*
FROM TableA A
FULL JOIN TableB B
ON A.id = B.id;
```

# Set Operations
## UNION
Combines results from two queries, removing duplicates.

```sql
SELECT column_name FROM TableA
UNION
SELECT column_name FROM TableB;
```

#UNION ALL
Combines results from two queries but keeps duplicates.

```sql
SELECT column_name FROM TableA
UNION ALL
SELECT column_name FROM TableB;

```

# INTERSECT
Returns only the rows that appear in both queries.

```sql
SELECT column_name FROM TableA
INTERSECT
SELECT column_name FROM TableB;
```

# EXCEPT (or MINUS in some databases)
Returns rows from the first query that do not exist in the second query.

```sql
SELECT column_name FROM TableA
EXCEPT
SELECT column_name FROM TableB;
```

# Primary Key vs Foreign Key

## **Primary Key**
A **Primary Key (PK)** is a unique identifier for each row in a table. It ensures that no two rows have the same value and that the column does not contain NULL values.

### **Characteristics:**
- Uniquely identifies each record.
- Cannot have NULL values.
- Each table can have only **one** primary key.
- Can consist of a **single column** or **multiple columns** (composite key).

### **Example:**
```sql
CREATE TABLE Students (
    student_id INT PRIMARY KEY,
    name VARCHAR(50),
    age INT
);
```

# Foreign Key
A Foreign Key (FK) is a column (or set of columns) in one table that references the primary key of another table. It establishes a relationship between tables.

# Characteristics:
- Links two tables together.
- Ensures referential integrity (a record in the child table must have a corresponding record in the parent table).
- Can contain duplicate values.
- Can have NULL values (unless specified otherwise).

### Example:

```sql
CREATE TABLE Enrollments (
    enrollment_id INT PRIMARY KEY,
    student_id INT,
    course_id INT,
    FOREIGN KEY (student_id) REFERENCES Students(student_id)
);
```

* Here, student_id in the Enrollments table is a foreign key that references student_id in the Students table.


| Feature         | Primary Key | Foreign Key |
|---------------|------------|------------|
| **Purpose** | Uniquely identifies a row | Establishes a relationship between tables |
| **Uniqueness** | Must be unique | Can have duplicate values |
| **NULL Values** | Not allowed | Allowed (unless constrained) |
| **Number per Table** | Only one primary key | Multiple foreign keys allowed |
| **Modification** | Cannot be changed | Can be updated (with cascading rules) |
| **Table Location** | Defined in the same table | Defined in a different table referencing the primary key |


In [ ]:
# SQL Concepts with Python SQLite Examples

## **1. Join Types**

### **INNER JOIN**
# Returns only matching rows from both tables.

import sqlite3

# Connect to database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create tables
cursor.execute("CREATE TABLE Students (id INTEGER PRIMARY KEY, name TEXT)")
cursor.execute("CREATE TABLE Enrollments (id INTEGER PRIMARY KEY, student_id INTEGER, course TEXT, FOREIGN KEY(student_id) REFERENCES Students(id))")

# Insert data
cursor.executemany("INSERT INTO Students VALUES (?, ?)", [(1, "Alice"), (2, "Bob")])
cursor.executemany("INSERT INTO Enrollments VALUES (?, ?, ?)", [(1, 1, "Math"), (2, 2, "Science"), (3, 1, "History")])

# INNER JOIN
cursor.execute("""
SELECT Students.name, Enrollments.course
FROM Students
INNER JOIN Enrollments ON Students.id = Enrollments.student_id
""")
print(cursor.fetchall())  # [('Alice', 'Math'), ('Bob', 'Science'), ('Alice', 'History')]


[('Alice', 'Math'), ('Bob', 'Science'), ('Alice', 'History')]


In [ ]:
def print_table(cursor):
    rows = cursor.fetchall()

    columns = [desc[0] for desc in cursor.description]
    for column in columns:
        print(column, end="\t")
    print()
    print("-" * 10 * len(columns))

    # Print table rows
    for row in rows:
        for item in row:
            print(item, end="\t")
        print()

    print()
    print()

In [ ]:
cursor.execute("""
SELECT Students.name, Enrollments.course
FROM Students
LEFT JOIN Enrollments ON Students.id = Enrollments.student_id
""")

print_table(cursor)


name	course	
--------------------
Alice	History	
Alice	Math	
Bob	Science	




In [ ]:
cursor.execute("""
SELECT Enrollments.course, Students.name
FROM Enrollments
LEFT JOIN Students ON Students.id = Enrollments.student_id
""")

print_table(cursor)


course	name	
--------------------
Math	Alice	
Science	Bob	
History	Alice	




In [ ]:
# Since RIGHT JOIN is not supported, we can rewrite it using a LEFT JOIN swap:

cursor.execute("""
SELECT Students.id, Students.name, Enrollments.course
FROM Students
LEFT JOIN Enrollments ON Students.id = Enrollments.student_id
UNION
SELECT Students.id, Students.name, Enrollments.course
FROM Enrollments
LEFT JOIN Students ON Students.id = Enrollments.student_id
""")


print_table(cursor)

id	name	course	
------------------------------
1	Alice	History	
1	Alice	Math	
2	Bob	Science	




In [ ]:
cursor.execute("""
SELECT name FROM Students
UNION
SELECT course FROM Enrollments
""")


print_table(cursor)

name	
----------
Alice	
Bob	
History	
Math	
Science	




In [ ]:
cursor.execute("""
SELECT name FROM Students
UNION ALL
SELECT course FROM Enrollments
""")


print_table(cursor)

name	
----------
Alice	
Bob	
Math	
Science	
History	




In [ ]:
cursor.execute("""
SELECT name FROM Students
INTERSECT
SELECT name FROM Students WHERE id = 1
""")


print_table(cursor)

name	
----------
Alice	




In [ ]:
cursor.execute("""
SELECT name FROM Students
EXCEPT
SELECT course FROM Enrollments
""")


print_table(cursor)

name	
----------
Alice	
Bob	




In [ ]:
# cursor.execute("CREATE TABLE Teachers (id INTEGER PRIMARY KEY, name TEXT)")
# cursor.execute("INSERT INTO Teachers VALUES (1, 'Dr. Smith')")
# cursor.execute("INSERT INTO Teachers VALUES (2, 'Dr. Jonathan')")

# cursor.execute("CREATE TABLE Classes (id INTEGER PRIMARY KEY, teacher_id INTEGER, FOREIGN KEY(teacher_id) REFERENCES Teachers(id))")
# cursor.execute("INSERT INTO Classes VALUES (1, 1)")

cursor.execute(f"SELECT * FROM Teachers")
print_table(cursor)
cursor.execute(f"SELECT * FROM Classes")
print_table(cursor)

id name 
--------------------
1 Dr. Smith 
2 Dr. Jonathan 


id teacher_id 
--------------------
1 1 




In [ ]:
conn.close()